# Development of miscellaneous new features for polymerist

## Testing polymerist importability within environment

In [ ]:
import polymerist as ps

print(ps.__version__)
print(ps.pascal(5))

In [ ]:
from polymerist.mdtools import openfftools

openfftools.POLYMERIST_TOOLKIT_REGISTRY.registered_toolkits

In [ ]:
import polymerist as ps
from polymerist.genutils.importutils import pyimports, module_hierarchy

print(module_hierarchy(ps))

In [ ]:
import nglview

print(nglview.__version__)
nglview.demo()

In [ ]:
from polymerist.polymers.monomers import specification
from rdkit import Chem

smi = 'CCO-c1ccccc1-N=C=C'
mol1 = Chem.MolFromSmiles(smi)
display(mol1)

sma = specification.expanded_SMILES(smi, assign_map_nums=True)
exp_sma = specification.compliant_mol_SMARTS(sma)
mol2 = Chem.MolFromSmarts(sma)
display(mol2)


In [ ]:
import numpy as np
import pandas as pd

from openff.toolkit import Molecule, Topology, ForceField

offmol = Molecule.from_smiles(smi)
offmol.generate_conformers(n_conformers=1)
offmol.visualize(backend='nglview')

## Experimenting with Protocols

In [ ]:
from typing import Protocol, runtime_checkable
from dataclasses import dataclass


@runtime_checkable
class Nominative(Protocol):
    def name(self) -> str:
        ...
        
class Foo(Nominative):
    def __init__(self, letter : str):
        self.letter = letter
        
    def name(self) -> str:
        return self.letter*3

@dataclass
class Bar:
    id : int
    
    def name(self) -> str:
        return f'{type(self).__name__}{self.id}'
    
@dataclass
class Baz:
    id : int
    
    def nombre(self) -> str:
        return 'name'

In [ ]:
classes = (Foo, Bar, Baz)
a = Foo('a')
b = Bar(2)
c = Baz(3)
instances = (a, b, c)

In [ ]:
for typ in classes:
    print(typ, issubclass(typ, Nominative))

In [ ]:
for inst in instances:
    print(inst, isinstance(inst, Nominative))

## Testing that reactant pathfinding for AnnotatedReaction works as intended

In [ ]:
from polymerist.rdutils.reactions.reactions import AnnotatedReaction
from polymerist.rdutils import rdkdraw

rdkdraw.disable_substruct_highlights()

from rdkit import Chem

In [ ]:
from rdkit import Chem

mol = Chem.MolFromSmiles('c1ccccc1C(=O)O')
mol

In [ ]:
bool(mol.GetAtomsMatchingQuery(Chem.MolFromSmarts('[#8]').GetAtomWithIdx(0)))

In [ ]:
rxn = AnnotatedReaction.from_smarts("[#8:1](-[*:2])-[H].[#17]-[#6:3](=[#8:4])-[*:5]>>[#8:1](-[*:2])-[#6:3](=[#8:4])-[*:5]")
# rxn = AnnotatedReaction.from_rxnfile('test.rxn')
reactant_templates_by_index = {i : reac_templ for i, reac_templ in enumerate(rxn.GetReactants())}

display(rxn)
print(reactant_templates_by_index)

In [ ]:
from polymerist.polymers.monomers.specification import expanded_SMILES

reactant_smiles = [
    'CC(O)C(Cl)C(Cl)CCO',
    'C1CCC(Cl)CC1O',
    'C1CCC(C(=O)Cl)CC1',
    'FC(F)(F)F',
    'c1(O)c(O)c(C(=O)Cl)c(O)c(C(=O)Cl)c1',
]

reactants = []
for smiles in reactant_smiles:
    exp_smiles = expanded_SMILES(smiles, assign_map_nums=False)
    mol = Chem.MolFromSmiles(exp_smiles, sanitize=False)
    Chem.SanitizeMol(mol)
    display(mol)
    reactants.append(mol)

In [ ]:
from itertools import combinations
from polymerist.rdutils.reactions.reactors import Reactor


reactor = Reactor(rxn)
for i, rs in enumerate(combinations(reactants, 2)):
    print(f'PAIR {i+1}')
    (react1, react2) = rs
    reactant_order = rxn.valid_reactant_ordering(rs)
    if reactant_order is None:
        print('NO ORDER FOUND:')
    else:
        print('ORDER FOUND:')
        print('+'*50)
        for m in reactant_order:
            display(m)
            
        display(reactor.react(rs)[0])

    print('='*50)

In [ ]:
from polymerist.rdutils.reactions.reactors import Reactor, PolymerizationReactor

rs = [reactants[0], reactants[4]]
reactor = PolymerizationReactor(rxn)

for adducts, frags in reactor.propagate(rs):
    print('adducts:')
    for adduct in adducts:
        display(adduct)
        
    print('fragments:')
    for frag in frags:
        display(frag)
        
    print('='*50)

# Another thing